# Gesichtserkennung und Charakterüberlagerung

## Gesichtserkennung (Face Detection) vs. Gesichtsidentifikation (Face Recognition)
- Gesichtserkennung : Erkennen von Gesichtsmerkmalen in Bildern oder Videos, um Gesichter zu finden
    - Beispiel : Temperaturmessung beim Betreten eines Gebäudes
- Gesichtsidentifikation : Bestimmen, wer die erkannten Gesichter sind
    - Beispiel : Anwesenheitskontrolle in Vorlesungen

Dieses Projekt erzielt das besseres Verständnis. Die unten stehenden Codes basieren auf die Referenz :  https://github.com/google/mediapipe/blob/master/docs/solutions/face_detection.md

### Package-Installation (anaconda prompt)
> pip install mediapipe

In [2]:
import cv2
import mediapipe as mp

# Variablen für das Finden und Markieren von Gesichtern definieren
mp_face_detection = mp.solutions.face_detection # Modul "face_detection" für Gesichtserkennung
mp_drawing = mp.solutions.drawing_utils # Modul "drawing_utils" zur Markierung der Gesichtsmerkmale

# Videodatei öffnen
cap = cv2.VideoCapture('face_video.mp4')

# Die with-Anweisung ermöglicht es, Dateien zu öffnen, ohne sie explizit mit close() schließen zu müssen
# Die automatische Freigabe der Ressourcen ist ein zusätzlicher Vorteil
with mp_face_detection.FaceDetection(model_selection = 0, min_detection_confidence = 0.7) as face_detection:
    # model_selection = 0 (Gesichter in 2m Entfernung) / 1 (Gesichter in 5m Entfernung)
    # min_detection_confidence : Mindestvertrauenswürdigkeit, um ein Gesicht zu erkennen (wie Threshold)
    
    while cap.isOpened(): # überprüfen, ob das Video erfolgreich geöffnet wurde
        success, image = cap.read()
    
        if not success:
            break

        # Zur Leistungsoptimierung kann das Bild optional als nicht schreibbar markert werden, um es per Referenz zu übergeben
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # mediapipe verwendet RGB-Farbformat
        results = face_detection.process(image) # Gesichter im Bild erkennen und es in "results" speichern

        # Die Gesichtserkennung im Bild zeichnen
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        if results.detections: # durch while-Schleife erkannte Gesichter im Bild zeichnen
            # 6 relevante Gesichtsmerkmale : rechtes Auge, linkes Auge, Nasenspitze, Mundmitte, rechtes Ohr, linkes Ohr
            for detection in results.detections:
                # mp_drawing.draw_detection(image, detection)
                # print(detection) # Informationen zu den in einem einzelen Frame erkannten Objekten
                
                # bestimmte Koordinaten abrufen
                keypoints = detection.location_data.relative_keypoints
                right_eye = keypoints[0]
                left_eye = keypoints[1]
                nose_tip = keypoints[2]
                
                h, w, _ = image.shape # Bildgröße (height, width, channel), Channel ist aber jetzt nicht gebraucht
                
                # Koordinaten der Gesichtsmerkmale im Bild (x, y)
                right_eye = (int(right_eye.x * w), int(right_eye.y * h)) 
                left_eye =  (int(left_eye.x * w), int(left_eye.y * h))
                nose_tip = (int(nose_tip.x * w), int(nose_tip.y * h))
                
                # Kreise um die Augen
                cv2.circle(image, right_eye, 50, (255, 0, 0), 10, cv2.LINE_AA) # blau
                cv2.circle(image, left_eye, 50, (0, 255, 0), 10, cv2.LINE_AA) # grün
                
                # Kreise um die Nase
                cv2.circle(image, nose_tip, 10, (0, 255, 255), 10, cv2.LINE_AA) # gelb
                
        cv2.imshow('MediaPipe Gesichtserkennung', cv2.resize(image, None, fx = 0.5, fy = 0.5))

        if cv2.waitKey(1) == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

## Bilderüberlagerung

In [5]:
import cv2
import mediapipe as mp

mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture('face_video.mp4')

# Bilder laden
image_right_eye = cv2.imread('right_eye.png') # 100 x 100
image_left_eye = cv2.imread('left_eye.png') # 100 x 100
image_nose = cv2.imread('nose.png') # 300 x 100 (width, height)

with mp_face_detection.FaceDetection(model_selection = 0, min_detection_confidence = 0.9) as face_detection:
    
    while cap.isOpened():
        success, image = cap.read()
    
        if not success:
            break

        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_detection.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        if results.detections:
            for detection in results.detections:
                
                keypoints = detection.location_data.relative_keypoints
                right_eye = keypoints[0]
                left_eye = keypoints[1]
                nose_tip = keypoints[2]
                
                h, w, _ = image.shape
                right_eye = (int(right_eye.x * w) - 20, int(right_eye.y * h) - 100) # Anpassen der Bildposition der Ohren
                left_eye =  (int(left_eye.x * w) - 20, int(left_eye.y * h) - 100)
                nose_tip = (int(nose_tip.x * w), int(nose_tip.y * h))
                
                # Bilder auf die jewieiligen Merkmale zeichnen
                image[right_eye[1] - 50 : right_eye[1] + 50, right_eye[0] - 50 : right_eye[0] + 50] = image_right_eye
                # Die Größe von "image_right_eye" beträgt 100 x 100,
                # und die Koordinaten von "right_eye" sind der Mittelpunkt des Auges,
                # daher wird der Bereich 50 Pixel darüber und darunter sowie 50 Pixel links und rechts davon (Viereck)ausgewählt
                image[left_eye[1] - 50 : left_eye[1] + 50, left_eye[0] - 50 : left_eye[0] + 50] = image_left_eye
                # Wenn man eine echte "solution" entwickeln würde, müsste man die Größe von "image_left_eye_ mit "image_left_eye.shape()" herausfinden
                image[nose_tip[1] - 50 : nose_tip[1] + 50, nose_tip[0] - 150 : nose_tip[0] + 150] = image_nose
                
                
        cv2.imshow('MediaPipe Charakterueberlagerung', cv2.resize(image, None, fx = 0.5, fy = 0.5))

        if cv2.waitKey(1) == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

Kostenlose Online-Bildbearbeitungswerkzeuge: https://pixlr.com

- image = cv2.imread('file_name', cv2.IMREAD_UNCHANGED)
    - Mit cv2.imread('file_name') allein erhält man nur width, height, channel. Um die Transparenz ebenfalls zu übernehmen, muss die Option "cv2.IMREAD_UNCHANGED verwendet werden.

- Fehler : could not broadcast (100, 100, 4) into (100, 100, 3)
    - Ein 4-Kanal-Bild kann nicht in ein 3-Kanal-Bild eingefügt werden.
        - Man muss die benötigten Kanäle separat extrahieren und einfügen
        - Eine "overlay()"-Funktion erstellen

#### overlay()-Funktion erstellen

In [6]:
import cv2
import mediapipe as mp

# ausführliche Erklärung zu "overlay()"-Funktion steht unten
def overlay(image, x, y, w, h, overlay_image): # Parameter: (Zielimage, x, y, width, height, Überlagerungsbild)
    alpha = overlay_image[:, :, 3] #BGRA
    mask_image = alpha / 255

    for c in range(0, 3): # channel BGR
        image[y - h : y + h, x - w : x + w, c] = (overlay_image[:, :, c] * mask_image) + (image[y - h : y + h, x - w : x + w, c] * (1 - mask_image))
        
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture('face_video.mp4')

image_right_eye = cv2.imread('right_eye_cat.png', cv2.IMREAD_UNCHANGED)
image_left_eye = cv2.imread('left_eye_cat.png', cv2.IMREAD_UNCHANGED)
image_nose = cv2.imread('nose_cat.png', cv2.IMREAD_UNCHANGED)

with mp_face_detection.FaceDetection(model_selection = 0, min_detection_confidence = 0.9) as face_detection:
    
    while cap.isOpened():
        success, image = cap.read()
    
        if not success:
            break

        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_detection.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        if results.detections:
            for detection in results.detections:
                
                keypoints = detection.location_data.relative_keypoints
                right_eye = keypoints[0]
                left_eye = keypoints[1]
                nose_tip = keypoints[2]
                
                h, w, _ = image.shape
                right_eye = (int(right_eye.x * w) - 20, int(right_eye.y * h) - 100)
                left_eye =  (int(left_eye.x * w) - 20, int(left_eye.y * h) - 100)
                nose_tip = (int(nose_tip.x * w), int(nose_tip.y * h))
                
                # overlay(image, x, y, w, h, overlay_image)
                overlay(image, *right_eye, 50, 50, image_right_eye)
                overlay(image, *left_eye, 50, 50, image_left_eye)
                overlay(image, *nose_tip, 150, 50, image_nose)
                
                
        cv2.imshow('MediaPipe verbesserte Charakterueberlagerung', cv2.resize(image, None, fx = 0.5, fy = 0.5))

        if cv2.waitKey(1) == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

#### Erklärung zu overlay_image (Überlagerungsbild)
- Das Überlagerungsbild (overlay_image) : 4 Kanäle (BGRA)
- Das Zielimage : 3 Kanäle (BGR), ohne alpha-Kanal für die Transparenz
- Das Ergebnis-Image : ddas Überlagerungsbild auf das Zielimage angemessen überlappen, um ein 3-Kanal-Bild zu erstellen

Wie funktioniert das?
- nur den alpha-Kanal (index = 3) aus dem Überlagerungsbild extrahieren [:, :, 3]
    - Der alpha-Wert liegt im Bereich 0 - 255 wie BGR
    - alpha / 255 ergibt mask_image mit Werten zwischen 0.0 und 1.0 (mask_image : Transparenz alpha)
    - 1 - mask_image ergibt die invertierte Transparenz, ebenfalls zwischen 1.0 - 0.0
    - mask_image und 1 - mask_image sind zueinander komplementär (mask_image + (1 - mask_image) = 1)
        - 1 : vollständig undurchsichtig
        - 0 : vollständig transparent
- Wenn das Überlagerungsbild 4-Kanal-BGRA ist und der alpha-Wert 1 ist, bedeutet das, dass es vollständig undurchsichtig ist und die Transparenz keine Rolle spielt
    - Das resultierende Bild wäre dann ebenfalls ein vollständig undurchsichtiges 3-Kanal-Bild, ohne alpha-Wert
    
Berechnung
- 3-Kanal-Ergebnis-Image BGR = (BGR aus 4-Kanal-Überlagerungsbild * mask_image) + (3-Kanal-Zielimage BGR * (1 - mask_image))
    - nur die BGR aus dem 4-Kanal-Überlagerungsbild verwenden
        - BGR aus dem 4-Kanal-Überlagerungsbild * mask_image
            - BGR * Transparenz
    - 3-Kanal-Zielimage BGR * (1 - mask_image)
        - Also, 3-Kanl-Zielimage BGR * (Wert zwischen 0 und 1)
            - BGR * invertierte Transparenz
- Beispiel 1 :
    - Überlagerungsbild 4-Kanal BGRA : Gelb, mask_image = 1.0 (vollständig undurchsichtiges Gelb)
        - (0, 255, 255) * 1.0 = (0, 255, 255) : gelbes Bilddatei unverändert
    - Zielimage 3-Kanal BGR : Rot
        - (0, 0, 255) * (1 - 1.0) = (0, 0, 255) * 0.0 = 0.0 : nicht-existiertes Datei
    - Ergebnis-Image BGR durch Berechnung: (0, 255, 255) + (0, 0, 0) = (0, 255, 255), Gelb, also Überlagerungsbild unverändert

- Beispiel 2 :
    - Überlagerungsbild 4-Kanal BGRA : Gelb, mask_image = 0.0 (vollständig transparentes Gelb)
        - (0, 255, 255) * 0.0 = 0.0
    - Zielimage 3-Kanal BGR : Rot
        - (0, 0, 255) * (1 - 0.0) = (0, 0, 255) * 1.0 = (0, 0, 255)
    - Ergebnis-Image BGR : (0, 0, 0) + (0, 0, 255) = (0, 0, 255), Rot, also Zielimage unverändert
    
- Beispiel 3 :
    - Überlagerungsbild 4-Kanal BGRA : Gelb, mask_image = 0.4 (transluzentes Gelb)
        - (0, 255, 255) * 0.4 = (0, 102, 102)
    - Zielimage 3-Kanal BGR : Rot
        - (0, 0, 255) * (1 - 0.4) = (0, 0, 255) * 0.6 = (0, 0, 153)
    - Ergebnis-Image BGR : (0, 102, 102) + (0, 0, 153) = (0, 102, 255), Orange, also Mischung aus Überlagerungsbild und Zielimage
    
Fazit :
- den alpha-Wert (Transparenz) des Überlagerungsbilds und den invertierten alpha-Wert (1 - mask_image) des Zielimages verwenden, um die richtigen Bildwerte zu berechnen
- das Überlagerungsbild und Zielimage mischen, entsprechend der Transparenz, um das Ergebnis-Image zu erstellen
    - mask_image = 1 : vollständig undurchsichtig, also das Überlagerungsbild unverändert verwenden
    - mask_image = 0 : vollständig transparent, also das Zielimage unverändert verwenden
    
Aktueller Code :  
image[y - h : y + h, x - w : x + w, c] = (overlay_image[:, :, c] * mask_image) + (image[y - h : y + h, x - w : x + w, c] * (1 - mask_image))  
- Ergebnis-Image 3-Kanal : image[y - h : y + h, x - w : x + w, c]
- BGR aus Überlagerungsbild 4-Kanal (overlay_image) : overlay_image[:, :, c]
- mask_image = alpha / 255
- Zielimage 3-Kanal : image[y - h : y + h, x - w : x + w, c]
- Für das Zielimage werden nur die Bereiche für height, width, channel verwendet. Vom Überlagerungsbild (overlay_image) werden nur die BGR-Kanäle verwendet, nicht den alpha-Kanal (A)
- Die Schleife for c in range(0, 3) entfernt automatisch den alpha-Kanal (c = 0, 1, 2)

Referenz: https://opencv-python.readthedocs.io/en/latest/